<a href="https://colab.research.google.com/github/seungyoonnam/machine-learning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

일반적인 모델 생성 방법
1. 훈련 세트로 모델 생성
2. 테스트세트로 평가 후 성능 평가 결과에 따라 재학습

위 과정 반복 후에는 결국 모델이 훈련세트뿐 아니라 테스트 세트에도 맞추어진다

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
#훈련세트와 검증세트를 sub_input, val_input으로 분리한다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)


#훈련세트를 분리한sub, val  값으로 성능평가
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


문제: 검증세트를 떼어낸만큼 훈련세트가 줄어든다. 결과 불안정

훈련 세트와 검증 세트의 평가 점수로 최적화

최적화 완료 후 테스트 세트로 최종평가

## 교차 검증
- 데이터를 k개의 폴드로 분할하고 각각 훈련세트와 검증 세트로 나누어 평가하는 방법을 k번 반복한다.
-default 값은 5-폴드

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00654626, 0.00642085, 0.00614834, 0.00597382, 0.00899124]), 'score_time': array([0.00084352, 0.00074863, 0.00066018, 0.00063014, 0.00123143]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의할점
- cross_validate()는 훈련세트를 랜덤하게 섞는 과정이 생략된 채로 순서대로 폴드를 나누어버린다. 분할기를 지정해줘야한다.
- 분류모델은 KFold 분할기, 회귀 모델은 StratifiedKFold 분할기를 사용한다.

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
#훈련세트를 섞은 뒤 10-폴드 교차검증을 수행한다.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
- 모델파라미터와 하이퍼파라미터: 하이퍼 파라미터는 모델이 학습할 수 없어 사람이 지정해줘야 하는 매개변수이다.
- 문제는 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안된다. 한꺼번에 찾아야 의미가 있다.



그리드서치
- 하이퍼파라미터 탐색과 교차검증을 한꺼번에 수행한다.
- 즉, cross_validate() 호출이 필요없다.

In [11]:
from sklearn.model_selection import GridSearchCV  #GridSearchCrossValidation

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
#최적의 하이퍼파라미터를 찾아 훈련
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #n_jobs: 사용할 CPU 코어 수. -1이면 모두 사용한다는 뜻

- GridSearchCV의 cv 매개변수 기본값은 5이므로 params에서 설정한 5개의 min_impurity_decrease 별로 결정 트리를 5번 수행.
- 즉, 25번의 결정 트리 모델을 훈련한다.

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

최고의 매개변수 조합의 모델은 gs.best_estimator_에 저장된다.

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


0.0001일때 최선이라는 뜻

In [18]:
#찾고자 하는 매개변수들을 지정(min_impurity_decrease, max_Depth, min_samples_split)
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),  #0.0001부터 시작해 0.001이 될때까지 0.0001씩 늘려가며
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치
- 두 번째 방법. 매개변수의 값의 범위나 간격을 예측하기 어려울 때,
- 매개변수 값의 목록이 아닌 매개변수를 샘플링할 수 있는 확률분포 객체를 전달한다.

In [22]:
from scipy.stats import uniform, randint

randint와 uniform
randint는 정수, uniform은 실수값을 선택

rvs: random variates

In [23]:
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 0, 7, 4, 6, 3, 6, 5, 3, 8])

10번의 샘플링으로는 부족해보인다.
1000번으로 재시도

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106, 102,  93,  84, 123,  94, 106,  77, 118,  97]))

In [25]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.95937125, 0.57931963, 0.08855403, 0.49332277, 0.62157753,
       0.03832697, 0.75591954, 0.1340932 , 0.99518239, 0.3225149 ])

4가지 매개변수에 대해 최적값을 찾아보자

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [27]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42) #n_iters: 위의 unofor/randint 범위 내에서 총 몇 번을 랜덤하게 뽑아 학습할 것인가?
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b62cd480>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a05b6323be0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b62cded0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b6322680>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [31]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b62cd480>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a05b6323be0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b62cded0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a05b6322680>},
                   random_state=42)

In [32]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
